In [1]:
import sys

ROOT_DIR = '..'
sys.path.append(ROOT_DIR)

In [2]:
import time

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from utils.evolution_functions import next_finite_drift_generation
from utils import selection_experiments as selection

In [ ]:
from utils.evolution_functions import generate_random_samples
from utils.statistic_utils import run_chi_squared_test

def run_exp(p0_list, q0_list, pop_size_list, n_loci, t, u=0, v=0, n=1):
    # Seleção Natural Primeiro
    # Na hora da cópia mutação
    # Migração riscada porque mutação bidirecional tem mesmo efeito da mutação bidimensional
    # Testar vários valores para observar taxas
    # 
    wAA = 1
    wAa = 0.9
    waa = 1
    ehw_gens = {}
    extinctions_gens = {}
    for p0, q0 in zip(p0_list, q0_list):
        for pop_size in pop_size_list:
            p_t_list = []
            q_t_list = []
            p2_t_list = []
            q2_t_list = []
            pq_t_list = []
            chi_squared_list = []
            population = np.array(["AA"] * int(p0 * p0 * pop_size) + ["aa"] * int(q0 * q0 * pop_size) + ["Aa"] * int(2 * p0 * q0 * pop_size))
            for pop in range(n_loci):
                p_t = np.zeros(t.shape)
                q_t = np.zeros(t.shape)
                chi_squared = np.zeros(t.shape[0] - 1)
                p_t[0] = p0
                q_t[0] = q0
                expected = generate_random_samples(p0, q0, pop_size)
                dict_key = f"N_pop={pop}, Pop_Size={pop_size} Pop={p0:.4f}x{q0:.4f}"
                for i in range(1, t.shape[0]):
                    # Instead of generating a population based on the p, q
                    # Generate p, q based on the population list
                    # 
                    selection.next_finite_gen(population, wAA, wAa, waa, pop_size)
                    p, q, population = next_finite_drift_generation(population, u, v, pop_size, seed=int(time.time()))
                    p_t[i] = p
                    q_t[i] = q

                    # Avaliar extinção
                    if p == 0 or q == 0 and not extinctions_gens.get(dict_key, None):
                        print(f"Extinction in gen {i}")
                        extinctions_gens[dict_key] = i
                    
                    # Valor de variância máxima -> Deriva máxima

                    observed = generate_random_samples(p, q, pop_size)
                    chi_2, is_correlated = run_chi_squared_test(observed, expected, n)
                    if not is_correlated and not ehw_gens.get(dict_key, None):
                        print(f"Not in EHW in gen {i}")
                        ehw_gens[dict_key] = i
                    
                    chi_squared[i - 1] = chi_2

                p_t_list.append(p_t)
                q_t_list.append(q_t)

                p2_t_list.append(p_t ** 2)
                q2_t_list.append(q_t ** 2)
                pq_t_list.append(2 * p_t * q_t)
                chi_squared_list.append(chi_squared)

                # Avaliar pontos de equilíbrio
                # plt.plot(t, p, label='Locus %d'%i)
            
            fig = plt.figure()
            ax = plt.subplot(111)

            for i, p in enumerate(p_t_list):
                ax.plot(t, p, label=f'p_{i+1}')

            # Shrink current axis by 20%
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

            # Put a legend to the right of the current axis
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            ax.title.set_text(f'Pop size: {pop_size}')
            plt.show()

            # plt.plot(t, p2_t_list, label='p²')
            # plt.plot(t, q2_t_list, label='q²')
            # plt.plot(t, pq_t_list, label='2pq')

            # plt.legend()
            # plt.show()

    if ehw_gens:
        for key, value in ehw_gens.items():
            print(f"End of EHW ({key}) Gen = {value}")

In [ ]:

run_exp([0.5], [0.5], [100], 1, np.arange(1000), u=0.0001, v=0.0001, n=1000)